# **Sarmad Ali**
# **10 Custom Queries**

### Query 1: Ranking Orders by Quantity per Customer

This query demonstrates how to apply ranking functions to partition and rank data based on customer orders in the **AdventureWorks2019** database. Specifically, it provides insights into each customer’s orders, ordered by **Order Quantity (OrderQty)**, and ranks them within each customer group.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    soh.CustomerID,
    sod.SalesOrderID,
    sod.OrderQty,
    RANK() OVER(PARTITION BY soh.CustomerID ORDER BY sod.OrderQty) AS rnk,
    DENSE_RANK() OVER(PARTITION BY soh.CustomerID ORDER BY sod.OrderQty) AS drnk
FROM 
    Sales.SalesOrderHeader AS soh
JOIN 
    Sales.SalesOrderDetail AS sod
ON 
    soh.SalesOrderID = sod.SalesOrderID;

### Query 2: Unique Unit Prices with Row Numbering

This query retrieves distinct **UnitPrice** values from the **SalesOrderDetail** table in the **AdventureWorks2019** database and assigns a unique row number to each price, ordered by **UnitPrice** in ascending order.


In [ ]:
USE Adventureworks2019
GO
SELECT DISTINCT UnitPrice, 
       ROW_NUMBER() OVER(ORDER BY UnitPrice) AS rownum
FROM Sales.SalesOrderDetail;

### Query 3: Order Quantity Differences with Previous and Next Quantities

This query calculates the difference between each **OrderQty** and its previous and next quantities for each **SalesOrderID** in the **SalesOrderDetail** table in the **AdventureWorks2019** database. It uses `LAG()` and `LEAD()` window functions to access the previous and next quantities, allowing for a comparison of each order with surrounding orders.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    SalesOrderID,
    OrderQty,
    LAG(OrderQty) OVER(ORDER BY SalesOrderID) AS prev_qty,
    LEAD(OrderQty) OVER(ORDER BY SalesOrderID) AS next_qty,
    OrderQty - LAG(OrderQty) OVER(ORDER BY SalesOrderID) AS diff_prev,
    OrderQty - LEAD(OrderQty) OVER(ORDER BY SalesOrderID) AS diff_next
FROM 
    Sales.SalesOrderDetail;

### Query 4: Sales Count by Year for Each Sales Person

This query counts the number of sales for each **SalesPersonID** by year (2012, 2013, and 2014) in the **SalesOrderHeader** table in the **AdventureWorks2019** database. It uses conditional aggregation to generate separate columns for each year, showing the count of orders per salesperson for each specified year.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    SalesPersonID,
    COUNT(CASE WHEN YEAR(OrderDate) = 2012 THEN 1 END) AS count2012,
    COUNT(CASE WHEN YEAR(OrderDate) = 2013 THEN 1 END) AS count2013,
    COUNT(CASE WHEN YEAR(OrderDate) = 2014 THEN 1 END) AS count2014
FROM 
    Sales.SalesOrderHeader
GROUP BY 
    SalesPersonID;

### Query 5: Running Total of TotalDue by Territory

This query calculates a running total of **TotalDue** for each **TerritoryID** in the **SalesOrderHeader** table in the **AdventureWorks2019** database. The running total is computed by summing **TotalDue** within each territory, ordered by **OrderDate**. This provides a cumulative total of sales due for each territory up to each order date.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    TerritoryID,
    SalesOrderID,
    OrderDate,
    TotalDue,
    SUM(TotalDue) OVER(PARTITION BY TerritoryID ORDER BY OrderDate 
                       ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningTotal
FROM 
    Sales.SalesOrderHeader
WHERE 
    TerritoryID IS NOT NULL
ORDER BY 
    TerritoryID, OrderDate;

### Query 6: Summing Order Quantities with Grouping Sets

This query calculates the sum of **OrderQty** for different grouping sets based on **SalesPersonID**, **CustomerID**, and **OrderYear** in the **AdventureWorks2019** database. It uses the `GROUPING SETS` clause to create multiple aggregations in a single query, along with `GROUPING_ID` to identify each grouping set in the result.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    GROUPING_ID(SalesPersonID, CustomerID, YEAR(OrderDate)) AS groupingset,
    SalesPersonID,
    CustomerID,
    YEAR(OrderDate) AS OrderYear,
    SUM(OrderQty) AS sumqty
FROM 
    Sales.SalesOrderHeader AS soh
JOIN 
    Sales.SalesOrderDetail AS sod
ON 
    soh.SalesOrderID = sod.SalesOrderID
GROUP BY 
    GROUPING SETS ((SalesPersonID, CustomerID, YEAR(OrderDate)), 
                   (SalesPersonID, YEAR(OrderDate)), 
                   (CustomerID, YEAR(OrderDate)), 
                   (YEAR(OrderDate)));

### Query 7: Ranking Sales Orders by TotalDue within Each Territory

This query ranks sales orders by **TotalDue** within each **TerritoryID** in the **SalesOrderHeader** table of the **AdventureWorks2019** database. It uses three ranking functions: `RANK()`, `DENSE_RANK()`, and `ROW_NUMBER()` to provide different types of rankings for **TotalDue** within each territory, ordered in descending order.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    TerritoryID,
    SalesOrderID,
    TotalDue,
    RANK() OVER(PARTITION BY TerritoryID ORDER BY TotalDue DESC) AS RankByTotalDue,
    DENSE_RANK() OVER(PARTITION BY TerritoryID ORDER BY TotalDue DESC) AS DenseRankByTotalDue,
    ROW_NUMBER() OVER(PARTITION BY TerritoryID ORDER BY TotalDue DESC) AS RowNumByTotalDue
FROM 
    Sales.SalesOrderHeader
WHERE 
    TerritoryID IS NOT NULL
ORDER BY 
    TerritoryID, TotalDue DESC;

### Query 8: Cumulative Percentage of TotalDue by Territory

This query calculates the cumulative percentage contribution of each **TotalDue** value within each **TerritoryID** in the **SalesOrderHeader** table of the **AdventureWorks2019** database. It uses a window function to compute the cumulative sum of **TotalDue** for each territory and expresses it as a percentage of the total **TotalDue** within the same territory.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    TerritoryID,
    SalesOrderID,
    TotalDue,
    100.0 * SUM(TotalDue) OVER(PARTITION BY TerritoryID ORDER BY TotalDue DESC) /
    SUM(TotalDue) OVER(PARTITION BY TerritoryID) AS CumulativePercentage
FROM 
    Sales.SalesOrderHeader
WHERE 
    TerritoryID IS NOT NULL
ORDER BY 
    TerritoryID, CumulativePercentage DESC;

### Query 9: Difference in TotalDue Between Current and Previous Orders

This query calculates the difference in **TotalDue** between each order and its previous order for each **CustomerID** in the **SalesOrderHeader** table of the **AdventureWorks2019** database. It uses the `LAG()` function to access the previous order’s **TotalDue**, allowing for a comparison to determine the change in total due between consecutive orders.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    CustomerID,
    SalesOrderID,
    OrderDate,
    TotalDue,
    LAG(TotalDue) OVER(PARTITION BY CustomerID ORDER BY OrderDate) AS PreviousTotalDue,
    TotalDue - LAG(TotalDue) OVER(PARTITION BY CustomerID ORDER BY OrderDate) AS Difference
FROM 
    Sales.SalesOrderHeader
ORDER BY 
    CustomerID, OrderDate;

### Query 10: Pivoting Total Sales by Year for Each Territory

This query uses the `PIVOT` operator to display the total **TotalDue** for each **TerritoryID** by year (2013, 2014, and 2015) in the **SalesOrderHeader** table of the **AdventureWorks2019** database. It pivots the data, creating separate columns for each year, which show the total sales per territory for each specified year.


In [ ]:
USE Adventureworks2019
GO
SELECT 
    TerritoryID,
    [2013] AS Sales2013,
    [2014] AS Sales2014,
    [2015] AS Sales2015
FROM 
    (SELECT TerritoryID, YEAR(OrderDate) AS OrderYear, TotalDue
     FROM Sales.SalesOrderHeader
     WHERE YEAR(OrderDate) BETWEEN 2013 AND 2015) AS SourceData
PIVOT 
    (SUM(TotalDue) FOR OrderYear IN ([2013], [2014], [2015])) AS PivotTable
ORDER BY 
    TerritoryID;